<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_fixed_v2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_fixed_v2.py

Parity classification on 4 bits:
1) Analytic CNOT cascade (100% accuracy)
2) Variational QNN with 5-fold CV
3) Quantum metric learning via trainable feature map
   - Vectorized kernel builder (no vdot)
   - step_and_cost to avoid ArrayBox→float
   - Final SVM with precomputed kernel
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Hyperparameters
n_bits     = 4
layers     = 2
n_steps    = 80
lr_kernel  = 0.2
svm_C      = 1.0
qnn_epochs = 30
qnn_lr     = 0.4

# 1) Data
X = pnp.array(list(itertools.product([0, 1], repeat=n_bits)))
y = pnp.array(np.sum(X, axis=1) % 2)  # parity labels {0,1}

# 2) Analytic CNOT cascade
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_analytic = [0 if analytic_circuit(x) > 0 else 1 for x in X]
acc_analytic   = accuracy_score(y, preds_analytic)
print(f"Analytic CNOT Cascade Accuracy: {acc_analytic:.3f}")

# 3) Variational QNN + 5-fold CV
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    # angle encode
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    # variational layer
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # entangle
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

y_signed = 1 - 2*y
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
var_scores = []

for train_idx, test_idx in kf.split(X, y):
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)

    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X[i]) for i in train_idx]
            return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
        params = opt.step(cost, params)

    preds_t = [qnn_circuit(params, X[i]) for i in test_idx]
    bits    = [0 if v > 0 else 1 for v in preds_t]
    var_scores.append(accuracy_score(y[test_idx], bits))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# 4) Quantum metric learning: vectorized Gram matrix
dev_feat = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_feat, interface="autograd")
def feature_map(params, x):
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    for layer in range(layers):
        for wire in range(n_bits):
            phi, theta, omega = params[layer, wire]
            qml.Rot(phi, theta, omega, wires=wire)
        for i in range(n_bits - 1):
            qml.CNOT(wires=[i, i + 1])
    return qml.state()

def build_kernel_matrix(params):
    # Stack all statevectors: shape (N, 2**n_bits)
    states = pnp.stack([feature_map(params, x) for x in X])
    re     = pnp.real(states)
    im     = pnp.imag(states)
    # compute real & imag overlaps via matrix multiplication
    real_overlap = re @ re.T + im @ im.T   # shape (N, N)
    imag_overlap = re @ im.T - im @ re.T
    return real_overlap**2 + imag_overlap**2

def kernel_target_alignment(K):
    y_s = 1 - 2*y
    T   = pnp.outer(y_s, y_s)
    num = pnp.sum(K * T)
    den = pnp.linalg.norm(K) * pnp.linalg.norm(T)
    return num / den

# initialize and optimize feature-map parameters
params_k = pnp.random.randn(layers, n_bits, 3, requires_grad=True) * 0.1
opt_k    = qml.GradientDescentOptimizer(stepsize=lr_kernel)

print("\nTraining feature-map via kernel–target alignment:")
for step in range(n_steps):
    def loss(p):
        K_mat = build_kernel_matrix(p)
        return -kernel_target_alignment(K_mat)
    params_k, curr_cost = opt_k.step_and_cost(loss, params_k)
    if step % 10 == 0:
        align = kernel_target_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:>2}: alignment = {align:.4f}")

# 5) Final SVM CV with trained kernel
K_final  = np.array(build_kernel_matrix(params_k))
kf       = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
ker_scores = []

for tr, te in kf.split(K_final, y):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_final[np.ix_(tr, tr)], y[tr])
    preds = svc.predict(K_final[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y[te], preds))

print(f"\nQuantum-Metric Kernel CV Accuracy: {np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}")

# 6) Visualize final kernel
plt.figure(figsize=(4, 4))
plt.title("Trained Kernel Matrix")
plt.imshow(K_final, cmap="viridis")
plt.colorbar()
plt.tight_layout()
plt.show()